In [1]:
## Import our dependencies

from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from config import password
import os
import pandas as pd
from sqlalchemy.ext.automap import automap_base
import datetime
import requests
import numpy as np


# First steps is to get a connection to the database

In [2]:
## Get a connection to the database and make the string into a variable

rds_connection_string = f'postgres:{password}@localhost:5432/Cryptocurrencies'

## Save this line for production connection to heroku database
# app.config['SQLALCHEMY_DATABASE_URI'] = os.environ.get('DATABASE_URL', '') or 

## Connect to local database

connection_string = f'postgresql://{rds_connection_string}'

## Create engine using the connection string to the database

engine = create_engine(connection_string)

# Reflect an existing database into a new model

Base = automap_base()

## reflect the tables

Base.prepare(engine, reflect=True)

In [3]:
## Find out the names of the tables available
# Base.metadata.tables

In [4]:
# Save a reference to the database tables we want to modify


bitcoin_four_years_data = Base.classes.bitcoin_four_years_data


In [10]:
btc_community = Base.classes.btc_community

In [ ]:
# session.query(bitcoin_four_years_data.id, bitcoin_four_years_data.name, bitcoin_four_years_data.dt).all()

# You can also make queries and updates to tables in database using sqlalchemy

In [24]:
# Use sqlalchemy if you want to do nosql

from sqlalchemy.orm import Session
session = Session(engine)

## Example of querying using sqlalchemy session

In [ ]:
# You can query using a session
#session.query(test_entry_order.name, test_entry_order.time).all()

## Example of adding records using SQLalchemy

In [ ]:
# # Let's add records using a session element

# session.add(test_entry_order(name='erin', time=8.31))

# # Commit and close the session for the changes to be uploaded to the databse

# session.commit()
# session.close()

## Example of queryng table using pandas and read_sql_query

In [ ]:
## Example of query using pandas
#pd.read_sql_query("select * from test_entry_order", con=engine)

## Example of adding records to a table using pandas

In [ ]:
## This method works for adding records to postgres database even if it returns an error

# pd.read_sql_query("insert into test_entry_order(name, time) values ('turt', 7.48)", con=engine)

## App that calculates how many missing dates there are between the table in question and today's date

### It returns a list with the dates missing from the table

In [ ]:
# Get today's date in the format required to make the request

def updated_dates_list(table_name):
    
    today = datetime.date.today()

    # Get the most recent date from data base using a query
    
    most_recent_entry = pd.read_sql_query(f"select dt from {table_name} order by dt desc limit 1", con=engine)['dt'][0]


    # Make list of dates between today and the most recent one, first get the number of days
    
    num_days = (today - most_recent_entry).days -1


    # Now make a list conprehension to provide the lists
    
    missing_dates_descending_order = [today - datetime.timedelta(days=x) for x in range(num_days)]
    missing_dates_ascending_order = [today - datetime.timedelta(days=num_days) + datetime.timedelta(days=x)for x in range(num_days+1)]
    
    ## Format the list to be apt for requests (Don't do this step here)
    #missing_dates = [missing_dates[x] for x in range(len(missing_dates))]
    
    return missing_dates_ascending_order

In [ ]:
missing_dates = updated_dates_list('bitcoin_four_years_data')
missing_dates

## App that makes requests for the missing dates to the API end point of coingecko

In [ ]:
## Make request for today's date


def get_data_missing_dates(index_start = 0, last_index_wanted = len(missing_dates), token = 'bitcoin'):
    
    append_to = []
    if len(missing_dates) == 0:
        print("Database up to date")

    for x in range(index_start, 1):

       
        
        date = {'date': missing_dates[x]}
        print("now checking:", missing_dates[x])
        
        url = f"https://api.coingecko.com/api/v3/coins/{token}/history?date={missing_dates[x].strftime('%d-%m-20%y')}localization=False"

        try:
            
            ## If the request is succesful, combine the date dictionary with the response json
    
            response_json = requests.get(url).json()
        
            print(response_json['symbol'])

            date.update(response_json)
            
            ## Append the combined dictionary to our desired list
        
            append_to.append(date)
    
            print('success')
        except:
            print(f"Looks like you have been timed out, continue at {x} for {missing_dates[x]}")
            
            
            break
            
#     return append_to




In [ ]:
missing_data = get_data_missing_dates()


In [ ]:
# Let's add records using a session element

for i in range(len(missing_data)):
    
    ## Since we have to add our own primary keys, findout max primary key
    
    last_primary_key = pd.read_sql_query("select max(id) from adding_bitcoin limit 10", con=engine)['max'][0].item()

    
    next_primary_key = last_primary_key+1

    
    session.add(adding_bitcoin(id = next_primary_key, dt = missing_data[i]['date'],
    name = 'bitcoin',
    symbol = missing_data[i]['symbol'],
    name_2 = missing_data[i]['name'],
    localization = f"{missing_data[i]['localization']}",
    image = f"{missing_data[i]['image']}",
    market_data = f"{missing_data[i]['market_data']}",
    community_data = f"{missing_data[i]['community_data']}",
    developer_data = f"{missing_data[i]['developer_data']}",
    public_interest_stats = f"{missing_data[i]['public_interest_stats']}"))

    # Commit and close the session for the changes to be uploaded to the databse

    session.commit()
    session.close()
print("SUCCESS!")
# session = Session(engine)

In [ ]:
pd.read_sql_query('SELECT dt from bitcoin_four_years_data', con=engine)

## Example for reversing the order of a csv file using pandas

In [ ]:
## 1. import the csv file as a datafram and save it the the variable df

#df = pd.read_csv('../static/data/bitcoin_four_years_data.csv').drop(columns='Unnamed: 0')

## 2. Convert the dataframe to a list

#bitcoin_four_years = df.values.tolist()

## 3. Reverse the order of the list

#bitcoin_four_years.reverse()

## Export the dataframe out as a csv 

#pd.DataFrame(bitcoin_four_years).to_csv("../static/data/reversed_bitcoin.csv")

In [31]:
def master_database_updating_app_bitcoin_only(table_name):
    
    #################################################################
    ## STEP 1: Get list of dates with missing data
    #################################################################
    
    # Get today's date
    
    today = datetime.date.today()

    # Get the most recent date from data base using a query
    
    most_recent_entry_date = pd.read_sql_query(f"select dt from {table_name} order by dt desc limit 1", con=engine)['dt'][0]


    ## Make list of dates between today and the most recent one, 
    ## First get the number of days
    
    num_days = (today - most_recent_entry_date).days -1


    # Now make a list conprehension to provide the lists
    
    missing_dates_descending_order = [today - datetime.timedelta(days=x) for x in range(num_days)]
    missing_dates_ascending_order = [today - datetime.timedelta(days=num_days) + datetime.timedelta(days=x)for x in range(num_days+1)]
    
    #################################################################
    ## STEP 2: Make requests to api for missing date from the table
    #################################################################
    
    
    # Get the name of the token
        
    token = pd.read_sql_query(f"select name from {table_name} order by id desc limit 1", con=engine)['name'][0]
    
    ## Check to see if database is up to date
    
    if len(missing_dates_ascending_order) == 0:
        print("Database up to date")

    for x in range(len(missing_dates_ascending_order)):
        
        session = Session(engine)
        
        dt = missing_dates_ascending_order[x]
        print("now checking:", missing_dates_ascending_order[x])
        
        url = f"https://api.coingecko.com/api/v3/coins/{token}/history?date={missing_dates_ascending_order[x].strftime('%d-%m-20%y')}localization=False"
        
        response_json = requests.get(url).json()
        
        #################################################################
        ## STEP 3: COLLECT THE RESPONSE AND ADD IT TO DATABASE
        #################################################################
        
        ## Since we have to add our own primary keys, find out max primary key number
    
        last_primary_key = pd.read_sql_query(f"select max(id) from {table_name} limit 10", con=engine)['max'][0].item()
        next_primary_key = last_primary_key+1

        ## Add our parameters to our session to update bitcoin_four_years_data
        
        id = next_primary_key 
        dt = dt
        name = token
        symbol = response_json['symbol']
        name_2 = response_json['name']
        localization = response_json['localization']
        image = response_json['image']
        market_data = response_json['market_data']
        community_data = response_json['community_data']
        developer_data = response_json['developer_data']
        public_interest_stats = response_json['public_interest_stats']
        
        session.add(bitcoin_four_years_data(id = id, 
        dt = dt,
        name = name,
        symbol = symbol,
        name_2 = name_2,
        localization = f'{localization}',
        image = f'{image}',
        market_data = f'{market_data}',
        community_data = f'{community_data}',
        developer_data = f'{developer_data}',
        public_interest_stats = f'{public_interest_stats}'))

        # Commit and close the session for the changes to be uploaded to the databse

       
        
        ##############################################################################
        ## STEP 4: UNPACK COMMUNITY DATA AND ADD IT TO ITS TABLE
        ##############################################################################
        
        facebook_likes = community_data['facebook_likes']
        twitter_followers = community_data['twitter_followers']
        reddit_average_posts_48h = community_data['reddit_average_posts_48h']
        reddit_average_comments_48h = community_data['reddit_average_comments_48h']
        reddit_subscribers = community_data['reddit_subscribers']
        reddit_accounts_active_48h = community_data['reddit_accounts_active_48h']
        
        session.add(btc_community(
        id = id,
        dt = dt,
        name = name,
        facebook_likes = facebook_likes,
        twitter_followers = twitter_followers,
        reddit_average_posts_48h = reddit_average_posts_48h,
        reddit_average_comments_48h = reddit_average_comments_48h,
        reddit_subscribers = reddit_subscribers,
        reddit_accounts_active_48h = reddit_accounts_active_48h
        ))
        
        session.commit()
        session.close()

        
    print("SUCCESS!")

In [28]:
session = Session(engine)

In [32]:
master_database_updating_app_bitcoin_only('bitcoin_four_years_data')

now checking: 2021-07-30
now checking: 2021-07-31
now checking: 2021-08-01
now checking: 2021-08-02
now checking: 2021-08-03
now checking: 2021-08-04
now checking: 2021-08-05
now checking: 2021-08-06
SUCCESS!
